Projeto desenvolvido no âmbito do módulo de Técnicas de Programação I, na trilha de Engenharia de dados do programa Santander Coders 2023, em parceria com a Ada Tech.

Projeto elaborado por:
- Andreza Lima
- Danilo Freitas
- Luis Felipe Gilisk
- Patrick Leite

# Descrição do projeto


Você trabalha em uma consultoria de dados que foi contratada para realizar a distribuição de materiais didáticos nas escolas da cidade do Rio de Janeiro. Sua missão é realizar tratamentos nos dados de acordo com as normas de padrão definidas pelo cliente e encontrar qual a melhor rota que um caminhão deve realizar para entregar os materiais didáticos de forma a otimizar o seu percurso. 

Para esse projeto você recebeu três arquivos:
- escolas.csv: contém os dados das escolas
- subprefeituras.csv: contém dados de quais bairros pertem a cada subprefeitura
- material_didatico.csv: contém a quantidade de material didático que cada escola deve receber

Como produto final, você deve entregar:
- um arquivo csv contendo os dados no padrão especificado com as escolas ordenadas em que os materiais didáticos devem ser entregues e com as colunas contendo id da escola, nome da escola, tipo da escola (EM, CIEP ou colégio), logradouro da entrega, número, bairro, subprefeitura, latitude, longitude e quantidade de material didático que deve ser entregue;
- um arquivo csv com a quantidade total de material escolar por subprefeitura para que sejam contabilizados os custos por subprefeitura

Como padrão dos dados, considere:
- nome das colunas em snake_case
- strings não devem conter acentos
- todas as strings devem estar em maiúsculo
- padronização do nome dos logradouros sem abreviação (Ex: R. deve estar como Rua)
- latitude e longitude devem conter apenas 5 casas decimais
- os ids da escola devem todos ser strings com 3 caracteres (Ex: '024')

Desafio:

Entregar um plot com a representação da melhor rota que você encontrou, por exemplo:
<img src='rota.png' width=600>


_______


# Proposta de resolução

## Importando pacotes

In [ ]:
## Executar apenas se for preciso instalar pacotes
# !pip install pandas
# !pip install unidecode

In [17]:
import pandas as pd
import unidecode 

## Funções auxiliares

In [97]:
# Função para tratamento de colunas strings (remove acentos, coloca em maiúsculo, remove abreviações)
def trata_string(df,*colunas):
    """
    Recebe um data frame e os nomes das colunas que devem ter seu conteúdo transformado
    Retorna o mesmo data frame com as colunas transformadas
    """
    for coluna in colunas:
        df[coluna] = [unidecode.unidecode(x) for x in df[coluna].str.upper() ]
    return df

## Tratamento de escolas.csv

### Conhecendo os dados

In [77]:
# Leitura do arquivo
escolas = pd.read_csv("escolas.csv")

In [99]:
# Verifica a estrutura
escolas.head()

,id,escolas_postos,bairro,endereco,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326"
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202"
2,483,EM CELESTINO SILVA,CENTRO,"R. do Lavradio, 56","-22,909293","-43,183579"
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,Praça Argentina 20,"-22,897629","-43,227456"
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,Praça Condessa Paulo de Frontin 45,"-22,924412","-43,208579"


In [78]:
# verifica informações mais detalhadas do data frame
escolas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              152 non-null    int64 
 1   Escolas_Postos  152 non-null    object
 2   BAIRRO          152 non-null    object
 3   ENDEREÇO        152 non-null    object
 4   lat             152 non-null    object
 5   lon             152 non-null    object
dtypes: int64(1), object(5)
memory usage: 7.2+ KB


Destes resultados, é possível verificar que:
- os nomes das colunas não estão em formato snake case
- existe um ç nos nomes das colunas
- o dataset contém 152 escolas
- nenhum campo contém nulos
- o campo id tem tipologia de número inteiro, quando pelas instruções deveria ser string com tamanho padronizado em 3
- os campos escolas_postos, bairro e endereço precisam ser tratados para:
    - não conter acentos
    - estarem todas em maiúsculo
    - remover abreviação
- os campos latitude e longitude devem ser tratados para conter apenas 5 casas decimais. (TRANFORMAR EM FLOAT?)

A partir desta constatação, serão executadas ações para corrigir os pontos identificados acima.

### Corrigindo nomes das colunas

In [104]:
## Renomeia os nomes das colunas para snake case

# transforma os nomes das colunas em minúsculos 
escolas.columns = escolas.columns.str.lower()

# remove espaços antes e depois dos nomes das colunas, se houver
escolas.columns = escolas.columns.str.strip()

# remove os acentos dos nomes das colunas
escolas.columns = [unidecode.unidecode(x) for x in escolas.columns ]

In [105]:
# verifica a nova estrutura
escolas.head()

,id,escolas_postos,bairro,endereco,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,Rua Carlos Seidl S/Nº,"-22,880888","-43,225326"
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,Rua Ébano 187,"-22,889574","-43,236202"
2,483,EM CELESTINO SILVA,CENTRO,"R. do Lavradio, 56","-22,909293","-43,183579"
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,Praça Argentina 20,"-22,897629","-43,227456"
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,Praça Condessa Paulo de Frontin 45,"-22,924412","-43,208579"


### Corrigindo Ids

In [63]:
# verifica id
escolas.id.describe()

count    152.000000
mean     342.638158
std      203.307667
min       11.000000
25%      160.000000
50%      346.500000
75%      520.750000
max      690.000000
Name: id, dtype: float64

Verifica-se que existem inteiros com menos de 3 caracteres, então além de transformar a coluna em string, será necessário incluir zeros à esquerda, quando necessário

In [89]:
# tranforma coluna id em string:
escolas["id"] = escolas.id.astype(str)

In [69]:
escolas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              152 non-null    object
 1   escolas_postos  152 non-null    object
 2   bairro          152 non-null    object
 3   endereco        152 non-null    object
 4   lat             152 non-null    object
 5   lon             152 non-null    object
dtypes: object(6)
memory usage: 7.2+ KB


Como pandas usa object dtype para armazenar strings, resta agora apenas adicionar os zeros.

In [90]:
# Inclui zeros até que a string tenha tamanho 3
escolas["id"] = escolas.id.str.zfill(3)

In [91]:
# Checando os novos ids
escolas.id.unique()

array(['178', '634', '483', '476', '132', '017', '055', '490', '600',
       '089', '387', '331', '450', '558', '413', '540', '309', '268',
       '523', '035', '011', '226', '638', '126', '180', '163', '361',
       '280', '060', '587', '653', '660', '382', '217', '378', '429',
       '025', '373', '624', '157', '020', '117', '502', '351', '135',
       '388', '440', '259', '422', '041', '537', '230', '379', '146',
       '566', '498', '663', '121', '161', '263', '118', '140', '260',
       '069', '090', '274', '676', '390', '608', '589', '256', '091',
       '248', '623', '603', '673', '520', '412', '031', '514', '023',
       '415', '016', '570', '528', '414', '690', '202', '550', '211',
       '206', '492', '466', '636', '527', '586', '080', '398', '175',
       '308', '576', '495', '641', '689', '171', '477', '687', '568',
       '399', '255', '262', '616', '320', '144', '534', '342', '052',
       '139', '323', '573', '560', '246', '319', '584', '311', '316',
       '497', '297',

### Tratando colunas do tipo string

In [108]:
trata_string(escolas,"escolas_postos","bairro","endereco")

,id,escolas_postos,bairro,endereco,lat,lon
0,178,CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL,CAJU,RUA CARLOS SEIDL S/NO,"-22,880888","-43,225326"
1,634,EM ALICE DO AMARAL PEIXOTO,BENFICA,RUA EBANO 187,"-22,889574","-43,236202"
2,483,EM CELESTINO SILVA,CENTRO,"R. DO LAVRADIO, 56","-22,909293","-43,183579"
3,476,ESCOLA MUNICIPAL FLORIANO PEIXOTO,SAO CRISTOVAO,PRACA ARGENTINA 20,"-22,897629","-43,227456"
4,132,EM PEREIRA PASSOS,RIO COMPRIDO,PRACA CONDESSA PAULO DE FRONTIN 45,"-22,924412","-43,208579"
...,...,...,...,...,...,...
147,474,EM ENGENHEIRO GASTAO RANGEL,GUARATIBA,ESTRADA DO MAGARCA 9.183,"-22,98046","-43,643545"
148,301,EM JONATAS SERRANO,GUARATIBA,"ESTRADA DO MATO ALTO, S/NO","-22,953163","-43,577409"
149,215,E.M. NARCISA AMALIA,ILHA DE GUARATIBA,ESTRADA TEODORETO DE CAMARGO S/N.O,"-23,009084","-43,537582"
150,606,EM PROFESSOR CASTILHO,ILHA DE GUARATIBA,CAMINHO DA MATRIZ 4406,"-22,994124","-43,593683"


Para padronizar as siglas, o conhecimento da área de negócio seria o mais poderoso insight para resolver todas as ocorrências. Mas na ausência deste conhecimento, será necessário mais algumas investigações.

In [112]:
# Verificando as entradas distintas para escolas_postos
escolas.escolas_postos.unique()

array(['CENTRO INTEGRADO DE EDUCACAO PUBLICA HENFIL',
       'EM ALICE DO AMARAL PEIXOTO', 'EM CELESTINO SILVA',
       'ESCOLA MUNICIPAL FLORIANO PEIXOTO', 'EM PEREIRA PASSOS',
       'E.M PRESIDENTE JOSE LINHARES', 'EM SANTA CATARINA',
       'E.M TIA CIATA', 'EM URUGUAI', 'EM MARIO CLAUDIO',
       'CIEP PRESIDENTE AGOSTINHO NETO', 'CIEP TANCREDO NEVES',
       'EM ALBERTO BARTH', 'EM CICERO PENA', 'EM COCIO BARCELOS',
       'EM HENRIQUE DODSWORTH', 'EM MEXICO',
       'ESCOLA MUNICIPAL MINAS GERAIS', 'EM PRESIDENTE DA COSTA E SILVA',
       'EM SENADOR CORREA', 'CIEP SAMUEL WEINER', 'EM AFRANIO PEIXOTO',
       'E.M AZEVEDO SODRE', 'EM DUQUE DE CAXIAS',
       'ESCOLA MUNICIPAL EQUADOR', 'EM LAUDIMIA TROTTA',
       'EM PRUDENTE DE MORAES',
       'ESCOLA MUNICIPAL GENERAL HUMBERTO DE SOUZA MELLO',
       'ESCOLA MUNICIPAL BARAO HOMEM DE MELO', 'EM ALMIRANTE TAMANDARE',
       'EM LUCIA MIGUEL PEREIRA', 'E.M NOSSA SENHORA DA POMPEIA',
       'EM SANTOS ANJOS', 'EM ALAGOAS', 'EM BR

In [113]:
# Verificando as entradas distintas para bairro
escolas.bairro.unique()

array(['CAJU', 'BENFICA', 'CENTRO', 'SAO CRISTOVAO', 'RIO COMPRIDO',
       'IPANEMA', 'SANTA TERESA', 'HUMAITA', 'CATETE', 'FLAMENGO',
       'COPACABANA', 'BOTAFOGO', 'URCA', 'LARANJEIRAS', 'TIJUCA',
       'ANDARAI', 'PRACA DA BANDEIRA', 'GRAJAU', 'VILA ISABEL',
       'MARACANA/ VILA ISABEL', 'MARACANA/ TIJUCA', 'VIDIGAL',
       'SAO CONRADO', 'TODOS OS SANTOS', 'LEBLON', 'PILARES',
       'AGUA SANTA', 'PIEDADE', 'RIACHUELO', 'MEIER', 'ENGENHO DE DENTRO',
       'ENGENHO NOVO', 'JACARE', 'DEL CASTILHO', 'INHAUMA',
       'PARADA DE LUCAS', 'MARIA DA GRACA', 'MANGUINHOS',
       'JARDIM AMERICA', 'PENHA CIRCULAR', 'OLARIA', 'CAMPO GRANDE',
       'PENHA', 'RAMOS', 'VIGARIO GERAL', 'BONSUCESSO', 'MADUREIRA',
       'ACARI', 'CACUIA', 'JARDIM GUANABARA', 'PARQUE COLUMBIA', 'PAVUNA',
       'ANCHIETA', 'GUADALUPE', 'TURIACU', 'VILA KOSMOS', 'CAVALCANTI',
       'VILA DA PENHA', 'REALENGO', 'OSWALDO CRUZ', 'ROCHA MIRANDA',
       'CASCADURA', 'BENTO RIBEIRO', 'COLEGIO', 'CIDADE DE DEU

In [114]:
# Verificando as entradas distintas para endereco
escolas.endereco.unique()

array(['RUA CARLOS SEIDL S/NO', 'RUA EBANO 187', 'R. DO LAVRADIO, 56',
       'PRACA ARGENTINA 20', 'PRACA CONDESSA PAULO DE FRONTIN 45',
       'R. BARAO DA TORRE, 90', 'RUA EDUARDO SANTOS 38',
       'AVENIDA PRESIDENTE VARGAS S/NO', 'RUA ANA NERI 192',
       'R. HADDOCK LOBO, 148', 'RUA VISCONDE DE SILVA S/NO',
       'RUA DO CATETE 77', 'AVENIDA OSWALDO CRUZ 124',
       'AV. ATLANTICA, 1976', 'R. BARAO DE IPANEMA, 34',
       'AVENIDA EPITACIO PESSOA 474', 'RUA DA MATRIZ 67',
       'AVENIDA PASTEUR 433', 'RUA ASSUNCAO, 257', 'R. SEN. CORREA',
       'AV HEITOR BELTRAO, S/NO', 'R. FERREIRA PONTES, 328',
       'RUA BARAO DE UBA 331', 'R. MAL. JOFRE, 74',
       'BOULEVARD 28 DE SETEMBRO, 353', 'R. ANTONIO BASILIO, 370',
       'RUA ENES DE SOUSA 36', 'R. OITO DE DEZEMBRO, 275',
       'R. ALM. JOAO CANDIDO BRASIL, 352',
       'AVENIDA PRESIDENTE JOAO GOULART 296',
       'RUA MARTAGAO GESTEIRA S/NDEG', 'R. CIRNE MAIA, 109',
       'RUA HUMBERTO DE CAMPOS S/NDEG', 'AV. DOM HELDER

Nesse caso, por exemplo, vê-se que existem siglas em escolas_postos:
* "EM", "E.M", "E.M" ou ainda "EM." parecem substituir "ESCOLA MUNICIPAL"
* "CIEP" parece substituir "CENTRO INTEGRADO DE EDUCAÇÃO PUBLICA".

Em bairro, existem alguns casos que contêm a string "MARACANA/ " antes do nome do bairro. Idealmente, teríamos uma lista oficial de bairros para fazer a comparação.


Em endereço, verificam-se os seguintes casos:
* "R." parece substituir "RUA"
* "AV" e "AV." parecem substituir "AVENIDA"
* "PCA." parece substituir "PRACA"
* "ESTR." parece substituir "ESTRADA"
* "S/NO", "S/N.O", "S/N", "S/NDEG", "S/N.DEG" (ESSES 2 ÚLTIMO EU NÃO CONHEÇO) são nomenclaturas que parecem significar "SEM NÚMERO"
* Em alguns casos, como "R. SEN. CORREA" e "R. MAL. JOFRE", existe siglas no nome da rua. Idealmente, teríamos uma base oficial de ruas em bairros para validar o nome da rua, mas como não temos a base nem o conhecimento das ruas no estado em questão, esses casos não serão tratados.
* Alguns têm vírgulas a separar o número do logradouro, outros não

Ressalta-se que só foi possível verificar cada caso porque se trata de uma base de dados pequena. Caso contrário, poderiam ser utilizadas técnicas de amostragem, nuvem de palavras, ou outra técnica para identificar e tratar as siglas.


## Tratamento de material_ditatico.csv

## Tratamento de subprefeituras.csv

## Criação dos bancos finais

## Desafio